# Terraform Output


In [1]:
GOOGLE_CLOUD_PROJECT = "stackoverflow-semantic-search"
GOOGLE_CLOUD_REGION = "asia-southeast1"

# Output from Terraform
GCS_BUCKET = "gs://stackoverflow-semantic-search-storage"
ARTIFACT_REGISTRY_REPO = (
    f"asia-southeast1-docker.pkg.dev/stackoverflow-semantic-search/stackoverflow-semantic-search-docker"
)
VPC_CONNECTOR = "vpc-conn"
SERVICE_ACCOUNT = "svc-acc@stackoverflow-semantic-search.iam.gserviceaccount.com"
VPC_NETWORK_NAME = "projects/stackoverflow-semantic-search/global/networks/stackoverflow-semantic-search-vpc-network"

VERTEX_AI_MODEL_ENDPOINT_RESOURCE = (
    "projects/500376812545/locations/asia-southeast1/endpoints/5296892868764368896"
)
VERTEX_AI_INDEX_ENDPOINT_RESOURCE = "projects/500376812545/locations/asia-southeast1/indexEndpoints/2734599767488200704"
DEPLOYED_INDEX_ID = "semantic_search_index"

# Create Redis Instance


In [3]:
REDIS_INSTANCE = f"{GOOGLE_CLOUD_PROJECT}-redis"

In [11]:
print(f'gcloud redis instances create --project {GOOGLE_CLOUD_PROJECT}',
      f'{REDIS_INSTANCE}',
      f'--tier basic',
      f'--size 2',
      f'--region {GOOGLE_CLOUD_REGION}',
      f'--redis-version redis_7_0',
      f'--network {VPC_NETWORK_NAME}',
      f'--connect-mode PRIVATE_SERVICE_ACCESS')

gcloud redis instances create --project stackoverflow-semantic-search stackoverflow-semantic-search-redis --tier basic --size 2 --region asia-southeast1 --redis-version redis_7_0 --network projects/stackoverflow-semantic-search/global/networks/stackoverflow-semantic-search-vpc-network --connect-mode PRIVATE_SERVICE_ACCESS


# Populate Redis Instance (Run within VPC Network in Notebook)


In [25]:
print('sudo apt-get install redis-tools -y')

sudo apt-get install redis-tools -y


In [5]:
REDIS_HOST = "10.6.6.3"
REDIS_PORT = "6379"

In [6]:
print(f'python bq_to_redis.py | redis-cli -h {REDIS_HOST} -p {REDIS_PORT} --pipe')

python bq_to_redis.py | redis-cli -h 10.6.6.3 -p 6379 --pipe


# Build Cloud Run Image


In [7]:
SERVICE_NAME = "semantic-search-be"

In [8]:
print(f'gcloud builds submit --gcs-source-staging-dir',
      f'{GCS_BUCKET}/cloudbuild/semantic-search-be/source',
      f'--gcs-log-dir {GCS_BUCKET}/cloudbuild/semantic-search-be/logs',
      f'--tag {ARTIFACT_REGISTRY_REPO}/{SERVICE_NAME}')

gcloud builds submit --gcs-source-staging-dir gs://stackoverflow-semantic-search-storage/cloudbuild/semantic-search-be/source --gcs-log-dir gs://stackoverflow-semantic-search-storage/cloudbuild/semantic-search-be/logs --tag asia-southeast1-docker.pkg.dev/stackoverflow-semantic-search/stackoverflow-semantic-search-docker/semantic-search-be


# Deploy Cloud Run Service


In [9]:
print(f'gcloud run deploy {SERVICE_NAME}',
      f'--image {ARTIFACT_REGISTRY_REPO}/{SERVICE_NAME}',
      f'--service-account {SERVICE_ACCOUNT}',
      f'--min-instances 1',
      f'--max-instances 4',
      f'--vpc-connector {VPC_CONNECTOR}',
      f'--allow-unauthenticated',
      f'--platform managed',
      f'--region {GOOGLE_CLOUD_REGION}',
      f'--project {GOOGLE_CLOUD_PROJECT}',
      f'--update-env-vars REDIS_HOST={REDIS_HOST},REDIS_PORT={REDIS_PORT},DEPLOYED_INDEX_ID={DEPLOYED_INDEX_ID},VERTEX_AI_INDEX_ENDPOINT_RESOURCE={VERTEX_AI_INDEX_ENDPOINT_RESOURCE},VERTEX_AI_MODEL_ENDPOINT_RESOURCE={VERTEX_AI_MODEL_ENDPOINT_RESOURCE}')

gcloud run deploy semantic-search-be --image asia-southeast1-docker.pkg.dev/stackoverflow-semantic-search/stackoverflow-semantic-search-docker/semantic-search-be --service-account svc-acc@stackoverflow-semantic-search.iam.gserviceaccount.com --min-instances 1 --max-instances 4 --vpc-connector vpc-conn --allow-unauthenticated --platform managed --region asia-southeast1 --project stackoverflow-semantic-search --update-env-vars REDIS_HOST=10.6.6.3,REDIS_PORT=6379,DEPLOYED_INDEX_ID=semantic_search_index,VERTEX_AI_INDEX_ENDPOINT_RESOURCE=projects/500376812545/locations/asia-southeast1/indexEndpoints/2734599767488200704,VERTEX_AI_MODEL_ENDPOINT_RESOURCE=projects/500376812545/locations/asia-southeast1/endpoints/5296892868764368896


# Delete GCP Resources


In [35]:
print(f'echo y | gcloud redis instances delete {REDIS_INSTANCE} --region={GOOGLE_CLOUD_REGION}')

echo y | gcloud redis instances delete stackoverflow-semantic-search-redis --region=asia-southeast1


In [36]:
print(f'echo y | gcloud run services delete {SERVICE_NAME} --region={GOOGLE_CLOUD_REGION}')

echo y | gcloud run services delete semantic-search-be --region=asia-southeast1
